# ETL Pipeline for Bison Data

This notebook cleans lab results data for various blood tests performed on wild bison in the State of Utah. Raw data is in .xlsx format with individual sheets for each test, having been extracted with Tabula from PDF results provided by the lab. Future improvements to this pipeline could include extracting directly from the PDF using python libraries. 

Clean data is organized into a single table, exported to .xlsx for use by the Utah Division of Wildlife Resources, and loaded into a multi-species SQLite database for future data analysis. 

In [1]:
#import libraries
import pandas as pd
import numpy as np
import re

In [2]:
#define handy functions

def panda_stripper(df):
    '''Strips all string columns in a pandas dataframe, in place. Seems like this should already be a pd method, but whatever.'''
    df_obj = df.select_dtypes(['object'])
    df[df_obj.columns] = df_obj.apply(lambda row: row.str.strip())
    return df

def get_sample_id(df, column):
    '''
    Extracts sample_id from a df column. 
    INPUT: df, column name
    Expected format: "BCB111 / BIS21-027 :: Serum"
    Action: splits string by whitespace or commas and returns first element of string
    OUTPUT: adds df column named 'sample_id' with the returned value in each cell
    '''
    df['sample_id'] = df[column].apply(lambda row: re.split(r"\s|,", row)[0])
    return df


## Read in Sample Sheet

The "Sample Sheet" provided by the DWR is the primary index of capture samples, and lab results data will be merged into this sheet based on the Sample ID.

In [3]:
#read sample sheet; this will be the final table 
bison_table = pd.read_excel('data/Bison_2021_22_Sample sheet.xlsx', usecols=[0, 1, 2, 3, 4, 5])

#give pythonic columns names
bison_table.columns = ['sample_id', 'archive_id', 'species', 'sex', 'capture_date', 'capture_unit']

#strip strings
bison_table = panda_stripper(bison_table)

In [4]:
bison_table.head()

,sample_id,archive_id,species,sex,capture_date,capture_unit
0,BCB1101,BIS21-018,Bison,Female,2022-02-27,Book Cliffs
1,BCB1102,BIS21-019,Bison,Female,2022-02-26,Book Cliffs
2,BCB1103,BIS21-020,Bison,Female,2022-02-27,Book Cliffs
3,BCB1104,BIS21-021,Bison,Female,2022-02-26,Book Cliffs
4,BCB1105,BIS21-022,Bison,Male,2022-02-26,Book Cliffs


## Lab Results

### BVD Tests

Two tests were performed for Bovine Diarrhea Virus: Type 1 and Type 2. 

In [5]:
#read excel table and strip strings
bvd_type1_df = pd.read_excel('data/bison_tables.xlsx', sheet_name='bd_diarrhea_type1a', usecols=[0,2])
bvd_type1_df = panda_stripper(bvd_type1_df)

#get sample_id
bvd_type1_df = get_sample_id(bvd_type1_df, 'Animals::Specimens')

#drop 'Animals::Specimens' column
del bvd_type1_df['Animals::Specimens']

#reorder columns
bvd_type1_df = bvd_type1_df.reindex(columns=['sample_id', 'Titer'])

#rename columns
bvd_type1_df.columns = ['sample_id', 'bvd_type1_result']

#merge into main table
bison_table = bison_table.merge(bvd_type1_df, on= 'sample_id')

In [6]:
#load bvd type 2 tests and strip strings
bvd_type2_df = pd.read_excel('data/bison_tables.xlsx', sheet_name='bv_diarrhea_type2', usecols= [0,2])
bvd_type2_df = panda_stripper(bvd_type2_df)

#get sample_id
bvd_type2_df = get_sample_id(bvd_type2_df, 'Animals::Specimens')

#drop 'Animals::Specimens' column
del bvd_type2_df['Animals::Specimens']

#reorder columns
bvd_type2_df = bvd_type2_df.reindex(columns= ['sample_id', 'Titer'])

#rename columns
bvd_type2_df.columns = ['sample_id', 'bvd_type2_result']

#merge into main table
bison_table = bison_table.merge(bvd_type2_df, on= 'sample_id')

### EHDV Tests

Epizootic Hemorhagic Disease Virus Tests 

In [7]:
#load data and strip
ehdv_df = pd.read_excel('data/bison_tables.xlsx', sheet_name='ehdv', usecols=[0,2])
ehdv_df = panda_stripper(ehdv_df)
ehdv_df.head()

,Animals::Specimens,Result
0,BCB1101 / BIS21-018,Positive
1,:: Serum,NaN
2,BCB1102 / BIS21-019,Negative
3,:: Serum,NaN
4,BCB1103 / BIS21-020,Negative


In [8]:
#drop extra rows
ehdv_df = ehdv_df.drop(ehdv_df[ehdv_df['Animals::Specimens'] == ':: Serum'].index)

#get sample_id
ehdv_df = get_sample_id(ehdv_df, 'Animals::Specimens')

#drop animals/specimens colums
del ehdv_df['Animals::Specimens']

#reorder columns
ehdv_df = ehdv_df.reindex(columns= ['sample_id', 'Result'])

#rename columns
ehdv_df.columns = ['sample_id', 'ehdv_result']

#merge into bison_table
bison_table = bison_table.merge(ehdv_df, on= 'sample_id')

### Bluetongue and Pregnancy

Bluetonge ELISA and Pregnancy ELISA tests are reported on the same table


In [9]:
#load and strip
bluepreg_df = pd.read_excel('data/bison_tables.xlsx', sheet_name='bluetongue')
bluepreg_df = panda_stripper(bluepreg_df)

In [ ]:
#name columns
bluepreg_df.columns = ['animal', 'preg_val', 'preg_result', 'bluetongue_result']

#get sample_id
bluepreg_df = get_sample_id(bluepreg_df, 'animal')

In [13]:

#drop animal col
del bluepreg_df['animal']

#merge into bison_table
bison_table = bison_table.merge(bluepreg_df, on= 'sample_id')

In [15]:
bison_table

,sample_id,archive_id,species,sex,capture_date,capture_unit,bvd_type1_result,bvd_type2_result,ehdv_result,preg_val,preg_result,bluetongue_result
0,BCB1101,BIS21-018,Bison,Female,2022-02-27,Book Cliffs,Negative @ 1:4,Negative @ 1:4,Positive,0.091,Not pregnant,Detected
1,BCB1102,BIS21-019,Bison,Female,2022-02-26,Book Cliffs,Negative @ 1:4,Negative @ 1:4,Negative,0.749,Pregnant,Not detected
2,BCB1103,BIS21-020,Bison,Female,2022-02-27,Book Cliffs,Negative @ 1:4,Negative @ 1:4,Negative,0.091,Not pregnant,Not detected
3,BCB1104,BIS21-021,Bison,Female,2022-02-26,Book Cliffs,Negative @ 1:4,Negative @ 1:4,Negative,0.718,Pregnant,Not detected
4,BCB1105,BIS21-022,Bison,Male,2022-02-26,Book Cliffs,Negative @ 1:4,Negative @ 1:4,Negative,0.089,Not pregnant,Not detected
5,BCB1106,BIS21-023,Bison,Female,2022-02-27,Book Cliffs,Negative @ 1:4,Negative @ 1:4,Negative,0.703,Pregnant,Not detected
6,BCB1107,BIS21-024,Bison,Female,2022-02-27,Book Cliffs,Negative @ 1:4,Negative @ 1:4,Positive,0.743,Pregnant,Detected
7,BCB1108,BIS21-025,Bison,Female,2022-02-27,Book Cliffs,Negative @ 1:4,Negative @ 1:4,Negative,0.681,Pregnant,Not detected
8,BCB1109,BIS21-026,Bison,Female,2022-02-26,Book Cliffs,Negative @ 1:4,Negative @ 1:4,Negative,0.088,Not pregnant,Not detected
9,BCB1110,BIS21-027,Bison,Female,2022-02-27,Book Cliffs,Negative @ 1:4,Negative @ 1:4,Negative,0.089,Not pregnant,Not detected


## Load

### Save to MS Excel file for DWR Vet Office

In [18]:
#bison_table.to_excel('data/finals/Bison 2021-2022 Lab Results.xlsx')